# Standard pipeline: analyzing 10K Multiome PBMC dataset from 10X genomics

## Introduction
In this tutorial we will extract the training data (profiles) for single-cell ATAC-seq data and RNA-seq data from Peripheral blood mononuclear cells (PBMCs).

## Import library and environment setup

In [3]:
import snapatac2_scooby as sp
import scanpy as sc

sp.__version__

'1.0.0'

We next need to download the bam files for RNA-seq, the fragment files for ATAC-seq and the count matrix.

In [4]:
# ! curl -O https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_gex_possorted_bam.bam
# ! curl -O https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_gex_possorted_bam.bam.bai
# ! curl -O https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz
# ! curl -O https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz.tbi
# ! curl -O https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5

## Read count matrix

In [5]:
adata_count = sc.read_10x_h5("pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5", gex_only=False)

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
adata_count

AnnData object with n_obs × n_vars = 2711 × 134920
    var: 'gene_ids', 'feature_types', 'genome'

## Read RNA Sequencing Data

We need to load and process our RNA sequencing bam files. This involves converting the raw BAM files into fragment files that SnapATAC can then read into the anndata format.

### Generating Fragment Files with SnapATAC

We have adapted snapATACs `sp.pp.make_fragment_file` function to also handle RNA-seq bam files that differ from ATAC-seq bam files in that they can contain split reads.

Here's a breakdown of the essential parameters and why they matter, specifically for RNA-seq data:

**1. Specifying Barcode and UMI Tags:**

- `barcode_tag`: This tells SnapATAC where to find the cell barcode information within each sequencing read. For data processed with 10x Genomics Cell Ranger, the default tag is `"CB"`. 
- `umi_tag`: This identifies the tag containing the Unique Molecular Identifier (UMI). SnapAtac automatically tries to remove PCR duplicates by filtering out reads that have exactly the same UMI and start and end value. For data processed with 10x Genomics Cell Ranger, the default tag is `"UB"`.

**2. Handling Read Strandedness:**

- `stranded`: Unlike ATAC-seq data, that usually are processed unstranded, RNA transcripts have a defined direction. Set `stranded=True` to create separate coverage tracks for reads mapping to the positive and negative strands of the genome that we will model as separate tracks.

**3. Addressing ATAC-Specific Parameters:**

- `is_paired`: Unlike most ATAC-seq reads, RNA-seq reads are not paired, so this argument needs to be set to `False`.
- `shift_left` and `shift_right`: These parameters are designed to correct for fragment shifts inherent to ATAC-seq data. For RNA-seq analysis, **it is essential to set both of these parameters to 0**.

**4. Leveraging High-Quality Reads (Cell Ranger > 3.0):**

- `xf_filter`: If your BAM files were aligned with Cell Ranger versions 3.0 or later, you'll find a helpful flag called "xf." This flag marks reads deemed to be of high quality by Cell Ranger's analysis pipeline. Setting `xf_filter=True` ensures that only these high-confidence reads contribute to your coverage tracks. This is **recommended** .  


In [7]:
%%time
sp.pp.make_fragment_file(
    bam_file="pbmc_granulocyte_sorted_3k_gex_possorted_bam.bam", 
    output_file="pbmc_granulocyte_sorted_3k_gex.fragments.bed.gz",
    barcode_tag="CB", 
    umi_tag="UB",
    umi_regex=None, 
    stranded=True, 
    is_paired=False, 
    shift_left=0, 
    shift_right=0, 
    xf_filter=True
)

CPU times: user 6min 59s, sys: 9.64 s, total: 7min 9s
Wall time: 7min 16s


{'frac_duplicates': 0.0,
 'frac_unmapped': 0.04059085115926564,
 'sequenced_reads': 164560538.0,
 'frac_valid_barcode': 0.9572655444283975,
 'frac_fragment_in_nucleosome_free_region': 0.9148212455488332,
 'frac_q30_bases_read2': nan,
 'sequenced_read_pairs': 0.0,
 'frac_q30_bases_read1': 0.8802099624070667,
 'frac_confidently_mapped': 0.9002277265282154,
 'frac_fragment_flanking_single_nucleosome': 0.0,
 'frac_nonnuclear': 0.10006990254249169}

The code creates two fragment files `pbmc_granulocyte_sorted_3k_gex.fragments.bed.minus.gz` and `pbmc_granulocyte_sorted_3k_gex.fragments.bed.plus.gz`, one for each strand.

### Read fragment files into AnnData format

We next want to read the fragment files into a AnnData object that stores the coverage information for each cell. Here we need to specify the genome that was used to align the data and the barcodes of cells that we would like to read. By default, we just use the barcodes of our filtered count matrix `adata_count`.

In [8]:
%%time
rna_coverage_plus = sp.pp.import_data(
                    "pbmc_granulocyte_sorted_3k_gex.fragments.bed.plus.gz", 
                    chrom_sizes=sp.genome.hg38, 
                    min_num_fragments=0, 
                    n_jobs=-1,
                    whitelist=adata_count.obs_names
                )
rna_coverage_minus = sp.pp.import_data(
                    "pbmc_granulocyte_sorted_3k_gex.fragments.bed.minus.gz", 
                    chrom_sizes=sp.genome.hg38, 
                    min_num_fragments=0, 
                    n_jobs=-1,
                    whitelist=adata_count.obs_names
                )

CPU times: user 10.9 s, sys: 384 ms, total: 11.2 s
Wall time: 6.34 s


In [9]:
assert (rna_coverage_plus.obs_names == adata_count.obs_names).all()

We now have the RNA coverage of both strands for each cell in the convenient adata format. The coverage is stored in `adata.obsm['fragment_single']`.

## Read ATAC Sequencing data

For ATAC data, we can usually directly start with the fragment files and read them into adata format. If you are directly starting with the fragment file and not the bam file, you want to set `sorted_by_barcode=False`. 

In [10]:
%%time
atac_coverage = sp.pp.import_data(
                "pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz", 
                chrom_sizes=sp.genome.hg38, 
                min_num_fragments=0, 
                n_jobs=-1,
                whitelist=adata_count.obs_names,
                sorted_by_barcode=False
            )

CPU times: user 1min 26s, sys: 2.27 s, total: 1min 29s
Wall time: 46.2 s


We next have to convert the fragments to insertion sites:

In [11]:
%%time
sp.pp.fragments_to_insertions(atac_coverage)

CPU times: user 1.19 s, sys: 78.6 ms, total: 1.27 s
Wall time: 1.29 s


We have now created three adata coverage files for RNA positive strand coverage, RNA negative strand coverage and ATAC insertions that we can use for training scooby!